In [1]:
import plotly.express as px
import pandas as pd
import json

pd.options.plotting.backend = "plotly"

In [2]:
data=pd.read_csv("dataset.js")

data

,day,month,year,cases,deaths,country,pop2019,continent,cum14daysCasesPer100000
0,14,12,2020,746,6,Afghanistan,38041757.0,Asia,9.01377925
1,13,12,2020,298,9,Afghanistan,38041757.0,Asia,7.05277624
2,12,12,2020,113,11,Afghanistan,38041757.0,Asia,6.86876792
3,11,12,2020,63,10,Afghanistan,38041757.0,Asia,7.13426564
4,10,12,2020,202,16,Afghanistan,38041757.0,Asia,6.96865815
...,...,...,...,...,...,...,...,...,...
61895,25,3,2020,0,0,Zimbabwe,14645473.0,Africa,-1
61896,24,3,2020,0,1,Zimbabwe,14645473.0,Africa,-1
61897,23,3,2020,0,0,Zimbabwe,14645473.0,Africa,-1
61898,22,3,2020,1,0,Zimbabwe,14645473.0,Africa,-1


In [3]:
the_dt=pd.to_datetime(data[["day","month","year"]])
the_dt

0       2020-12-14
1       2020-12-13
2       2020-12-12
3       2020-12-11
4       2020-12-10
           ...    
61895   2020-03-25
61896   2020-03-24
61897   2020-03-23
61898   2020-03-22
61899   2020-03-21
Length: 61900, dtype: datetime64[ns]

In [4]:
data["date"]=the_dt

In [5]:
data=data.drop(["day","month","year"],axis=1)
data

,cases,deaths,country,pop2019,continent,cum14daysCasesPer100000,date
0,746,6,Afghanistan,38041757.0,Asia,9.01377925,2020-12-14
1,298,9,Afghanistan,38041757.0,Asia,7.05277624,2020-12-13
2,113,11,Afghanistan,38041757.0,Asia,6.86876792,2020-12-12
3,63,10,Afghanistan,38041757.0,Asia,7.13426564,2020-12-11
4,202,16,Afghanistan,38041757.0,Asia,6.96865815,2020-12-10
...,...,...,...,...,...,...,...
61895,0,0,Zimbabwe,14645473.0,Africa,-1,2020-03-25
61896,0,1,Zimbabwe,14645473.0,Africa,-1,2020-03-24
61897,0,0,Zimbabwe,14645473.0,Africa,-1,2020-03-23
61898,1,0,Zimbabwe,14645473.0,Africa,-1,2020-03-22


In [6]:
df_grouped=data.groupby(by=["country",pd.Grouper(key="date",axis=0, freq="14D")]).agg({"cases":"sum","deaths":"sum","pop2019":"max","cum14daysCasesPer100000":"last"})
df_grouped

cases  deaths     pop2019 cum14daysCasesPer100000
country     date                                                         
Afghanistan 2019-12-31      0       0  38041757.0                      -1
            2020-01-14      0       0  38041757.0                       0
            2020-01-28      0       0  38041757.0                       0
            2020-02-11      0       0  38041757.0                       0
            2020-02-25      4       0  38041757.0              0.00262869
...                       ...     ...         ...                     ...
Zimbabwe    2020-10-06    262       3  14645473.0              1.31098531
            2020-10-20    227      12  14645473.0              1.78212066
            2020-11-03    412      14  14645473.0              1.57045116
            2020-11-17   1164      19  14645473.0              3.46864864
            2020-12-01   1296      31  14645473.0              7.76349115

[4532 rows x 4 columns]

In [7]:
df_grouped=df_grouped.drop(["cum14daysCasesPer100000"],axis=1)
df_grouped

cases  deaths     pop2019
country     date                                 
Afghanistan 2019-12-31      0       0  38041757.0
            2020-01-14      0       0  38041757.0
            2020-01-28      0       0  38041757.0
            2020-02-11      0       0  38041757.0
            2020-02-25      4       0  38041757.0
...                       ...     ...         ...
Zimbabwe    2020-10-06    262       3  14645473.0
            2020-10-20    227      12  14645473.0
            2020-11-03    412      14  14645473.0
            2020-11-17   1164      19  14645473.0
            2020-12-01   1296      31  14645473.0

[4532 rows x 3 columns]

In [8]:
df_grouped["proportionCases100000"]=round(df_grouped["cases"]/df_grouped["pop2019"]*100000,4)
df_grouped["proportionDeath100000"]=df_grouped["deaths"]/df_grouped["pop2019"]*100000
df_grouped["deaths/cases"]=round(df_grouped["deaths"]/df_grouped["cases"],4)
df_grouped=df_grouped.fillna(0)
df_grouped = df_grouped.where(df_grouped >= 0,0)
df_grouped

cases  deaths     pop2019  proportionCases100000  \
country     date                                                           
Afghanistan 2019-12-31      0       0  38041757.0                 0.0000   
            2020-01-14      0       0  38041757.0                 0.0000   
            2020-01-28      0       0  38041757.0                 0.0000   
            2020-02-11      0       0  38041757.0                 0.0000   
            2020-02-25      4       0  38041757.0                 0.0105   
...                       ...     ...         ...                    ...   
Zimbabwe    2020-10-06    262       3  14645473.0                 1.7889   
            2020-10-20    227      12  14645473.0                 1.5500   
            2020-11-03    412      14  14645473.0                 2.8132   
            2020-11-17   1164      19  14645473.0                 7.9478   
            2020-12-01   1296      31  14645473.0                 8.8492   

                        proportionDeath100000  deaths/cases  
country     date                                             
Afghanistan 2019-12-31               0.000000        0.0000  
            2020-01-14               0.000000        0.0000  
            2020-01-28               0.000000        0.0000  
            2020-02-11               0.000000        0.0000  
            2020-02-25               0.000000        0.0000  
...                                       ...           ...  
Zimbabwe    2020-10-06               0.020484        0.0115  
            2020-10-20               0.081937        0.0529  
            2020-11-03               0.095593        0.0340  
            2020-11-17               0.129733        0.0163  
            2020-12-01               0.211670        0.0239  

[4532 rows x 6 columns]

In [9]:
df_plot=df_grouped.reset_index()

In [10]:
data.continent.unique()

array(['Asia', 'Europe', 'Africa', 'America', 'Oceania', 'Other'],
      dtype=object)

In [11]:
df_plot=df_plot[df_plot["date"]!="2019-12-31"]
df_plot["date"]=df_plot["date"].astype(str)

In [12]:
# df_plot.loc[1,"continent"]=data.loc[data["country"]==df_plot.loc[1, "country"], "continent"]

In [13]:
df_plot["continent"]=0
for index in df_plot.index:
    df_plot.loc[index,"continent"]=data.loc[data["country"]==df_plot.loc[index, "country"], "continent"].unique()[0]

df_plot

,country,date,cases,deaths,pop2019,proportionCases100000,proportionDeath100000,deaths/cases,continent
1,Afghanistan,2020-01-14,0,0,38041757.0,0.0000,0.000000,0.0000,Asia
2,Afghanistan,2020-01-28,0,0,38041757.0,0.0000,0.000000,0.0000,Asia
3,Afghanistan,2020-02-11,0,0,38041757.0,0.0000,0.000000,0.0000,Asia
4,Afghanistan,2020-02-25,4,0,38041757.0,0.0105,0.000000,0.0000,Asia
5,Afghanistan,2020-03-10,30,0,38041757.0,0.0789,0.000000,0.0000,Asia
...,...,...,...,...,...,...,...,...,...
4527,Zimbabwe,2020-10-06,262,3,14645473.0,1.7889,0.020484,0.0115,Africa
4528,Zimbabwe,2020-10-20,227,12,14645473.0,1.5500,0.081937,0.0529,Africa
4529,Zimbabwe,2020-11-03,412,14,14645473.0,2.8132,0.095593,0.0340,Africa
4530,Zimbabwe,2020-11-17,1164,19,14645473.0,7.9478,0.129733,0.0163,Africa


In [14]:
df_plot["colors"]=df_plot["continent"]
df_plot["colors"]=df_plot["colors"].replace({"Asia":"red","America":"white","Europe":"blue","Oceania":"green","Africa":"brown","Other":"yellow"})
df_plot

,country,date,cases,deaths,pop2019,proportionCases100000,proportionDeath100000,deaths/cases,continent,colors
1,Afghanistan,2020-01-14,0,0,38041757.0,0.0000,0.000000,0.0000,Asia,red
2,Afghanistan,2020-01-28,0,0,38041757.0,0.0000,0.000000,0.0000,Asia,red
3,Afghanistan,2020-02-11,0,0,38041757.0,0.0000,0.000000,0.0000,Asia,red
4,Afghanistan,2020-02-25,4,0,38041757.0,0.0105,0.000000,0.0000,Asia,red
5,Afghanistan,2020-03-10,30,0,38041757.0,0.0789,0.000000,0.0000,Asia,red
...,...,...,...,...,...,...,...,...,...,...
4527,Zimbabwe,2020-10-06,262,3,14645473.0,1.7889,0.020484,0.0115,Africa,brown
4528,Zimbabwe,2020-10-20,227,12,14645473.0,1.5500,0.081937,0.0529,Africa,brown
4529,Zimbabwe,2020-11-03,412,14,14645473.0,2.8132,0.095593,0.0340,Africa,brown
4530,Zimbabwe,2020-11-17,1164,19,14645473.0,7.9478,0.129733,0.0163,Africa,brown


In [15]:
px.colors.qualitative.D3

['#1F77B4',
 '#FF7F0E',
 '#2CA02C',
 '#D62728',
 '#9467BD',
 '#8C564B',
 '#E377C2',
 '#7F7F7F',
 '#BCBD22',
 '#17BECF']

In [16]:
df_plot=df_plot.drop(df_plot[df_plot["continent"]=="Other"].index,axis=0)

In [17]:
the_html_plot=df_plot.plot.scatter(y="deaths/cases",
                                   x="proportionCases100000",
                                   animation_frame="date",
                                   animation_group="country",
                                   hover_name="country",
                                   render_mode="svg",
                                   color="continent",
                                #    color_discrete_sequence=[df_plot["colors"].unique()],
                                    # color_discrete_map="identity",
                                #    color_discrete_map={"Asia":"rgb(255,0,0)","America":"rgb(0,255,0)","Europe":"rgb(0,0,255)","Oceania":"rgb(255,0,255)","Africa":"rgb(255,255,0)"},
                                    # color_discrete_sequence=[df_plot["colors"].to_list()],
                                #    symbol="continent",
                                   title="Analisi della situazione pandemica nel mondo del 2020",
                                   labels={"proportionCases100000":"Casi positivi ogni 100.000 abitanti",
                                           "deaths/cases":"Tasso di mortalità"})
# the_html_plot.update_layout(xaxis=dict(showline=True, zeroline=True))
the_html_plot=the_html_plot.update_xaxes(showline=True,
                           linecolor = "black")
the_html_plot=the_html_plot.update_yaxes(showline=True,
                           linecolor = "black")

                        #     .add_annotation(
                        #        showarrow=True
                        #    )
the_html_plot

In [55]:

fig = px.scatter(df_plot, 
                 y="deaths/cases",
                 x="proportionCases100000", 
                 color="continent",
                 hover_name="country",
                 animation_frame="date",
                #  animation_group="country",
                 symbol="continent",
                 color_discrete_sequence=["#FF331F", "#AF00E0", "#0025F5", "#10E0D4", "#2FCC37"],
                 title="<b>Analisi della situazione pandemica del mondo nell'anno 2020</b>",
                 labels={"proportionCases100000":"Casi positivi ogni 100.000 abitanti",
                                           "deaths/cases":"Tasso di mortalità"},
                  # marginal_x="rug", 
                  # marginal_y="rug",
                  # trendline="ols"
                )
# fig.update_layout(
#     xaxis=dict(autorange=True),
#     yaxis=dict(autorange=True)
# )
fig.update_layout(
    font_family="Noto Sans",
    font_color="black",
    title_font_family="Noto Sans",
    title_font_color="black",
    legend_title_font_color="gray",
    legend_title="Continenti",
    legend=dict(orientation="h",
                yanchor="bottom",
                y=1.02,
                xanchor="right",
                x=1),
    paper_bgcolor="#e8ddcc",
    plot_bgcolor="#e3e3e3"
    
)

fig.show()

In [56]:
fig.write_html("plotFinale.html")

len(df_plot[df_plot['date']=='2020-05-05'].country.unique())
# len(df_plot.date.unique())

import plotly.express as px
import pandas as pd

df = pd.DataFrame({
    'x': [1, 2, 3, 4],
    'y': [10, 20, 30, 40],
    'frame': [1, 2, 3, 4]
})

fig = px.scatter(df, x='x', y='y', animation_frame='frame', animation_group='x')

fig.update_layout(
    xaxis=dict(autorange=True),
    yaxis=dict(autorange=True)
)

fig.show()

df_plot[df_plot["country"]=="Liechtenstein"]

df_plot[df_plot["country"]=="San_Marino"]

import plotly.express as px
import pandas as pd

df = pd.DataFrame({
    "Date": ["2022-01-01", "2022-01-02", "2022-01-03", "2022-01-04", "2022-01-05"],
    "Country": ["USA", "USA", "USA", "Canada", "Canada"],
    "Value1": [10, 20, 30, 40, 50],
    "Value2": [100, 200, 300, 400, 500]
})

# Group the data by country and date
grouped = df.groupby(["Country", "Date"])

# Get the index of the next value1, value2 for each group
next_index = grouped.apply(lambda x: x.index[1] if len(x) > 1 else x.index[0])

# Get the next value1, value2 for each group
next_value1 = df.loc[next_index, "Value1"].tolist()
next_value2 = df.loc[next_index, "Value2"].tolist()

# Calculate the position of each arrow
x0 = df["Value1"].tolist()
y0 = df["Value2"].tolist()
x1 = next_value1
y1 = next_value2
x = [(x0[i] + x1[i])/2 for i in range(len(df))]
y = [(y0[i] + y1[i])/2 for i in range(len(df))]

fig = px.scatter(df, x="Value1", y="Value2", color="Country")

# Add the annotations for each arrow
for i in range(len(df)):
    fig.add_shape(
        type="line",
        x0=x0[i],
        y0=y0[i],
        x1=x1[i],
        y1=y1[i],
        xref="x",
        yref="y",
        line=dict(
            color="gray",
            width=1,
            dash="dot"
        ),
        # arrow=dict(
        #     arrowhead=2,
        #     arrowsize=1,
        #     ax=x1[i]-x0[i],
        #     ay=y1[i]-y0[i]
        # )
    )

fig.show()